# Calculate Per-Language Counts of Thankers and Superthankers
July 25, 2019

Columns needed for MailChimp:
* user_email: email address for the account
* anonymized_id: anonymized ID
* user_name: Wikipedia username
* lang: language Wikipedia

In [11]:
import csv, os, glob
import pandas as pd

### Load Superthankers

In [45]:
data_dir = "/home/civilservant/Tresors/CivilServant/projects/wikipedia-integration/gratitude-study/Data Drills/thanker_hardlaunch/"
superthankers = {
    "fa":[],
    "de":[],
    "pl":[]
}
stl = []
with open(os.path.join(data_dir, "SuperThankers-07.25.2019.csv")) as f:
    for row in csv.DictReader(f):
        superthankers[row['wiki']].append(row['\ufeffusername'])

liason_usernames = []
with open(os.path.join(data_dir, "liasons-07.29.2019.csv")) as f:
    for row in csv.DictReader(f):
        liason_usernames.append(row['user_name'])

### Create a lookup table from consent records

In [54]:
id_key = dict()

for lang in superthankers.keys():
    with open(os.path.join(data_dir, "julia_input", lang + "_consent_6_26_clean.csv")) as f:
        for row in csv.DictReader(f):
            id_key[row['ID']] = {
                "user_name": row['user_name'],
                "user_email": row['user_email']} 

### Load merged survey and observational data

In [56]:
thanker_ids = []

for lang in superthankers.keys():
    matched_keys = 0
    unmatched_keys = 0
    superthanker_count = 0
    thanker_count = 0
    anon_ids = []
    
    with open(os.path.join(data_dir, "historical_survey_merged", lang + "-merged-20190729.csv")) as f:
        for row in csv.DictReader(f):
            if(row['anonymized_id'] in id_key.keys()):
                matched_keys += 1
                
                ##tally up the results
                anon_id = row['anonymized_id']
                user_name = id_key[anon_id]['user_name']
                user_email    = id_key[anon_id]['user_email']
                if user_name in superthankers[lang] or user_name in liason_usernames:
                    superthanker_count += 1
                else:
                    thanker_count += 1
                    thanker_ids.append({
                        "user_name":user_name,
                        "user_email": user_email,
                        "anon_id": anon_id,
                        "lang": lang
                    })
                    anon_ids.append(anon_id)
            else:
                unmatched_keys += 1
                print(row['anonymized_id'])
        print("{0} PARTICIPATION COUNTS".format(lang))
        print("")
        print("  {0} thankers".format(thanker_count))
        print("  {0} superthankers (that consented)".format(superthanker_count))
                
        print("  DIAGNOSTIC:")
        print("    {0} Matched Keys: {1}".format( lang, matched_keys))
        print("    {0} Unmatched Keys: {1}".format( lang, unmatched_keys))
        print("    {0} Unique anonymized IDs".format(len(set(anon_ids))))
        print("")

fa PARTICIPATION COUNTS

  53 thankers
  2 superthankers (that consented)
  DIAGNOSTIC:
    fa Matched Keys: 55
    fa Unmatched Keys: 0
    53 Unique anonymized IDs

de PARTICIPATION COUNTS

  294 thankers
  5 superthankers (that consented)
  DIAGNOSTIC:
    de Matched Keys: 299
    de Unmatched Keys: 0
    294 Unique anonymized IDs

pl PARTICIPATION COUNTS

  64 thankers
  2 superthankers (that consented)
  DIAGNOSTIC:
    pl Matched Keys: 66
    pl Unmatched Keys: 0
    64 Unique anonymized IDs



# Output Anonymized IDs of Thankers For Recruitment

In [57]:
print("{0} total thankers in the study".format(len(thanker_ids)))

411 total thankers in the study


In [59]:
pd.DataFrame(thanker_ids).to_csv(os.path.join(data_dir,
        "thanker_email_recruitment", "thanker_recruitment_emails-07.29.2019.csv"))